## EM and KMeans++ for finding the individual distributions of a Gaussian Mixture Model


### Including Libraries used

In [0]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm

### Some Utility Functions

In [0]:
def error(p1, p2):
    ans = 0
    for i in range(len(p1)):
        ans += dist(p1[i], p2[i])
    ans1 = 0
    p2=p2[::-1] # This is required as in kmeans++ the order of the centers may be different
    for i in range(len(p1)):
        ans1 += dist(p1[i], p2[i])
    return min(ans,ans1)

In [0]:
def dist(p1,p2):
    p1, p2 = np.array(p1), np.array(p2)
    return np.sqrt(np.sum((p1-p2)**2))

In [0]:
def gauss(x,mu,sig):
    dim = len(x)
    return np.exp( -((np.matrix(x-mu)) @ (sig.I) @ np.matrix(x-mu).T)/2 )/(np.sqrt(((2*np.pi)**dim)*np.linalg.det(sig)))

### Function to generate datasets

In [0]:
def create_dataset(c, k):
    np.random.seed(52)
    cen = c*np.random.normal(0,1)
    mu1 = np.array([0.0]*32)
    mu2 = np.array([cen]*32)
    mus = [mu1,mu2]
    data = []
    for i in range(k):
        sel_ind = np.random.choice([0,1],p = [0.5, 0.5])
        point = np.random.multivariate_normal(mus[sel_ind], np.eye(32))
        data.append(point)
    return data, mus

### Expectation Maximization

In [0]:
def EM(data, n_iter, k):
    np.random.seed(52)
    # Initialization
    n = len(data)
    dim = len(data[0])
    mu = [np.zeros(dim), np.ones(dim)]
    sig = [np.matrix(np.eye(dim)) for i in range(k)]
    w = np.array([0.5]*k)
    theta = np.array([[0.0 for i in range(k)] for j in range(n)])
    for it in range(n_iter):
        # E Step
        for i in range(n):
            total = [w[l]*gauss(data[i],mu[l],sig[l]) for l in range(k)]
            total_sum = np.sum(total)
            for j in range(k):
                theta[i,j] = total[j]/total_sum

        # M Step
        N = [np.sum([theta[i,j] for i in range(n)]) for j in range(k)]
        for j in range(k):
            w[j] = N[j]/np.sum(N)
            mu[j] = np.zeros(dim)
            for i in range(n):
                mu[j] += theta[i,j]*data[i]
            mu[j] = mu[j]/N[j]
            sig[j] = np.zeros([dim,dim])
            for i in range(n):
                sig[j] += theta[i,j]*( np.matrix(data[i]-mu[j]).T @ np.matrix(data[i]-mu[j]) )
            sig[j] = np.matrix(sig[j]/N[j])
    return mu

### KMeans++

In [0]:
def kmpp(data, n_iter, k):
    np.random.seed(52)
    # Initialization
    n = len(data)
    cents = [np.random.choice(list(range(n)))]
    dists = [dist(data[i], data[cents[-1]]) for i in range(n)]
    for i in range(1,k):
        npsum = np.sum([dists[j]**2 for j in range(n)])
        sel = np.random.choice(list(range(n)), p = [dists[j]**2/npsum for j in range(n)])
        cents.append(sel)
        for j in range(len(dists)):
            dists[j] = min(dists[j], dist(data[j], data[cents[-1]]))
            
    # Assigning to clusters
    clus = [[] for i in range(k)]
    for i in range(n):
        dis = 10**9+7
        cen = -1
        for j in range(k):
            dis1 = dist(data[i],data[cents[j]])
            if dis1<dis:
                dis = dis1
                cen = j
        clus[cen].append(i)
    
    # Lloyd's Algorithm
    while n_iter+1:
        # Finding Current Centers
        centers = []
        for i in range(k):
            centers.append(np.mean([data[j] for j in clus[i]], axis=0))
        cents = centers
        # Reassigning centers
        clus = [[] for i in range(k)]
        for i in range(n):
            dis = 10**9+7
            cen = -1
            for j in range(k):
                dis1 = dist(data[i],cents[j])
                if dis1<dis:
                    dis = dis1
                    cen = j
            clus[cen].append(i)
        
        n_iter -= 1
    return cents

### Generating Dataset

In [0]:
cs = [0.5, 1, 1.5, 2, 3, 4, 8]
Data = []
MUs = []

In [0]:
for c in cs:
    data, mu = create_dataset(c, 10000)
    Data.append(data)
    MUs.append(mu)

### Running EM algorithm on the data generated 

In [0]:
em_outs = []
calc_mus = []

In [0]:
for _ in range(len(cs)):
    calc_mu = EM(Data[_], 50, 2)
    calc_mus.append(calc_mu)
    err = error(MUs[_], calc_mu)
    em_outs.append(err)

### Running KMeans++ algorithm on the data generated

In [0]:
kmpp_outs = []
calc_mus_ = []
for _ in range(len(cs)):
    calc_mu = kmpp(Data[_], 5, 2)
    calc_mus_.append(calc_mu)
    kmpp_outs.append(error(MUs[_], calc_mu))

### Displaying Output

In [13]:
OUT1 = pd.DataFrame(pd.Series(cs))
OUT1 = pd.concat([OUT1, pd.Series(em_outs)],axis=1)
OUT1.columns = ["c", "error"]
print(OUT1)

     c     error
0  0.5  2.013632
1  1.0  0.247600
2  1.5  0.188911
3  2.0  0.186185
4  3.0  0.184561
5  4.0  0.184558
6  8.0  0.184558


In [14]:
OUT2 = pd.DataFrame(pd.Series(cs))
OUT2 = pd.concat([OUT2, pd.Series(kmpp_outs)],axis=1)
OUT2.columns = ["c", "error"]
print(OUT2)

     c     error
0  0.5  0.977011
1  1.0  0.238308
2  1.5  0.194850
3  2.0  0.187138
4  3.0  0.184558
5  4.0  0.184558
6  8.0  0.184558
